In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import box
from descartes.patch import PolygonPatch
import ipywidgets as widgets
import ipywidgets.widgets.interaction
#from ipywidgets import interactive, fixed
from ipywidgets import *
import random
import pandas as pd

import re

from shapely import geometry
import pickle
import time

from ipyparallel import Client

from shapely import affinity
from shapely.geometry import LineString
import warnings
warnings.simplefilter('ignore', np.RankWarning)



In [2]:
os.chdir('/home/charly/Lab/Projects/uJ/local/experiments/HT-LB/')
#os.chdir('/home/esb/Projects/uJ/uJ_master/experiments/HT-Sine2hr/')


In [3]:
pwd()

'/media/charly/Data/Lab/Projects/uJ/local/experiments/HT-LB'

In [4]:
#sys.path.insert(0, '/Users/ESB/RPM_SYNC/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/esb/Projects/uJ/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/Users/ESB/SYNC_RPM/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/charly/Lab/Projects/uJ/uJ_master/current/uJ_src/python/')
sys.path.insert(0, '/home/charly/Lab/Projects/uJ/local/current/uJ_src/python/')




In [5]:
from readroi import *
from DataManagers import *
from TrackingFunctions import *
from PlottingFunctions import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!
TrackingFunctions... loaded!
PlottingFunctions... loaded!


In [ ]:
# import sys, importlib
# importlib.reload(sys.modules['DataManagers'])
# importlib.reload(sys.modules['TrackingFunctions'])
# from DataManagers import *
# from TrackingFunctions import *

## Note 
### Start a cluster with ipcluster start

## Experiment Parameters

In [6]:

expeLabel="HT13-LB"
rootDir="../../data_sample/HT13-LB/"
trap_list="xy02,xy03,xy04,xy05,xy06,xy10".split(",")  
trap="xy04" 

#Input files
dirNameROIs=rootDir+"data_rois/"+trap+"/DsRed+GFP/"
dirNameDATA=rootDir+'data/relativeIntensity/'+trap+"/"

#Output
dirNameCELLS=rootDir+'data_cells/'+trap+'/DsRed+GFP/'
if not os.path.exists(dirNameCELLS):
    print("making dir cells...")
    os.makedirs(dirNameCELLS)

dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/DsRed+GFP/'
if not os.path.exists(dirNameTRACKEDCELLS):
    print("making dir tracked cells...")
    os.makedirs(dirNameTRACKEDCELLS)

    
    
dirNameTRACKEDCELLS_IMGS=rootDir+'images_cells_tracked/'+trap+'/DsRed+GFP/'
if not os.path.exists(dirNameTRACKEDCELLS_IMGS):
    print("making dir tracked cells imgs ...")
    os.makedirs(dirNameTRACKEDCELLS_IMGS)

dirNameDATALINEAGES=rootDir+'data/'
if not os.path.exists(dirNameDATALINEAGES):
    print("making dir lineages...")
    os.makedirs(dirNameDATALINEAGES)

#toFile=False
toFile=True

#Parameters 
bin_num=41
bin_max =0.75
bin_min=-.75
bin_size = (bin_max-bin_min)/bin_num; 

frame2min=5
frame_experiment_start=1;
frame_signal_start=int(865-(frame_experiment_start-1)/2)
frame_signal_end=int(865-(frame_experiment_start-1)/2)
frame_experiment_end=int(865-(frame_experiment_start-1)/2)

t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min


frames=range(int(frame_experiment_start),int(frame_experiment_end+1))

times=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]
#print("\nMinutes:  %s"%times)
#print("\nHours:  [%s,%s]"%(times_hour[0],times_hour[-1]))


print("\nFrames:  %s\t %s->%s hours"%(frames,times_hour[0],times_hour[-1]))
print("t_experiment_start=%s\t(%sh)"%(t_experiment_start, t_experiment_start/60))
print("t_experiment_end=%s\t(%sh)"%(t_experiment_end, t_experiment_end/60))


#Prepare file system
dataPath="%sdata/DsRed+GFP/"%rootDir

#if not os.path.exists(figurePathDM) and toFile:
#    os.mkdir(figurePathDM)



making dir cells...
making dir tracked cells...
making dir tracked cells imgs ...

Frames:  range(1, 866)	 0.0->72.0 hours
t_experiment_start=0	(0.0h)
t_experiment_end=4320	(72.0h)


### Save tracked cells

In [ ]:
file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
file_cells.sort()
for frame, this_file in enumerate(file_cells):
    this_cells=cells_tracked[frame] 
    #print(frame, this_file)
    fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
#    print(fileNameFrameCells)
    save_cells(this_cells, fileNameTrackedCells)

### Load tracked cells

In [ ]:
#_tracked_cells=cells
cells_tracked=load_cells(dirNameTRACKEDCELLS)

### (Here we should correct tracking)

In [ ]:
def plot_this_cells(matchID,cellID_list,local_cells):
    
    #fig, ax = plt.subplots(1, 2, sharey=True, figsize=(30,30))
    fig, ax = plt.subplots(1, 2, figsize=(15,6))
    
    #ax = plt.axes()
    ax[0].set_aspect('equal')
    ax[1].set_aspect('equal')
   # plt.xlim(0,640)
    plt.ylim(0,512)
#     #plt.axis('off')
#     ax[0].xlim(0,640)
#     ax[0].ylim(0,512)
#     ax[1].xlim(0,640)
#     ax[1].ylim(0,512)
#     
    ax[0].clear()
    ax[1].clear()
    matchID=matchID.split('.')
    frame=int(matchID[0])
    index=int(matchID[1])
    this_cell=local_cells[frame][index]
    this_poly=this_cell['roiPoly']
    #print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
    #patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.3, zorder=2)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.7, zorder=2)
    
    ax[0].add_patch(patch)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=1)
    ax[1].add_patch(patch)
    this_axis=this_cell['axis']
    linexy=np.array(this_axis)
    linex=linexy[:,0]
    liney=linexy[:,1]
    ax[1].plot(linex,liney,'r*',alpha=0.5)
    ax[0].plot(linex,liney,'c-',alpha=0.5)
    
    for this_cell in local_cells[frame]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax[0].add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax[0].plot(linex,liney,'y-',alpha=0.5)
    
    
    for this_cell in local_cells[frame+1]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax[1].add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax[1].plot(linex,liney,'y-',alpha=0.5)
    
    
    
    for this_id in cellID_list:
        this_id=this_id.split('.')
        frame=int(this_id[0])
        index=int(this_id[1])
        this_cell=local_cells[frame][index]
        this_poly=this_cell['roiPoly']
       # print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0.2,0.2,0.2], alpha=0.7, zorder=3)    
        
        
        
        ax[1].add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax[1].plot(linex,liney,'c-',alpha=0.5)
    

    
    text=ax[0].text(0,0, "", va="bottom", ha="left")
    ax[0].set_title('frame: %s cellID: %s'%(frame-1,matchID))
    ax[1].set_title('frame: %s cellIDs: %s'%(frame,cellID_list))    
   # ax[0].axis('off')
   # ax[1].axis('off')
    #plt.show()
    #rfig=fig
    #plt.close()
    #plt.show()

    

In [ ]:
cont=0
to_correct_plots=[]
for this_cells in cells_tracked:
    for this_cell in this_cells:
        if(len(this_cell['trackedBy_next_frame'])>2):
            cont+=1
            print(cont,end='\r')
            print(this_cell['cellID'],"\t",this_cell['trackedBy_next_frame'])
            this_fig=plot_this_cells(this_cell['cellID'],this_cell['trackedBy_next_frame'],cells_tracked)
            to_correct_plots.append(this_fig)
            if(cont>=5):
                break
    if(cont>=5):
        break
cont     

In [ ]:
to_correct_plots[1]

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
 display: none;
}
</style>

In [ ]:
def get_frame_polys(this_tracked_cells,frame):
    this_polys=[]
    for i,this_cell in enumerate(this_tracked_cells[frame-1]):
        this_poly=this_cell['roiPoly']
        this_polys.append(this_poly)


    return(this_polys)
            
            
            
            
def plot_this_cells2(matchID,cellID_list,local_cells):
    
    fig=plt.figure(figsize=(15,6))
    #fig, ax = plt.subplots(1, 2, )
    ax0=fig.add_subplot(121)
    ax1=fig.add_subplot(122)
    #ax = plt.axes()
    ax0.set_aspect('equal')
    ax1.set_aspect('equal')
  # plt.xlim(0,640)
   # plt.ylim(0,512)
#     #plt.axis('off')
#     ax[0].xlim(0,640)
#     ax[0].ylim(0,512)
#     ax[1].xlim(0,640)
#     ax[1].ylim(0,512)
#     
    ax0.clear()
    ax1.clear()
    
    matchID=matchID.split('.')
    frame=int(matchID[0])
    index=int(matchID[1])
    this_cell=local_cells[frame][index]
    this_poly=this_cell['roiPoly']
    #print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
    #patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.3, zorder=2)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.7, zorder=2)
    
    ax0.add_patch(patch)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=1)
    ax1.add_patch(patch)
    this_axis=this_cell['axis']
    linexy=np.array(this_axis)
    linex=linexy[:,0]
    liney=linexy[:,1]
    ax1.plot(linex,liney,'r*',alpha=0.5)
    ax0.plot(linex,liney,'c-',alpha=0.5)
    
    for this_cell in local_cells[frame]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax0.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax0.plot(linex,liney,'y-',alpha=0.5)
    
    
    for this_cell in local_cells[frame+1]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax1.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax1.plot(linex,liney,'y-',alpha=0.5)
    
    
    
    for this_id in cellID_list:
        this_id=this_id.split('.')
        frame=int(this_id[0])
        index=int(this_id[1])
        this_cell=local_cells[frame][index]
        this_poly=this_cell['roiPoly']
       # print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0.2,0.2,0.2], alpha=0.7, zorder=3)    
        
        
        
        ax1.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax1.plot(linex,liney,'c-',alpha=0.5)
    

    
    
    ax0.set_title('frame: %s cellID: %s'%(frame-1,matchID))
    ax1.set_title('frame: %s cellIDs: %s'%(frame,cellID_list))    
    
   # ax[0].axis('off')
   # ax[1].axis('off')
    #plt.show()
    #rfig=fig
    #plt.close()
    #plt.show()
    return fig
    

In [ ]:
print(len(this_frame_polys))

In [ ]:
%matplotlib notebook
#%matplotlib inline   
#if plot does not show,uncomment this line run, comment and re-run

def onclickSelect(event):
    tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
    
    this_point = Point(event.xdata,event.ydata)
    tx="x=%s y=%s"%(this_point.x,this_point.y)
    
    for select_index, select_cell in enumerate(cells_tracked[this_frame]):
        select_poly=select_cell['roiPoly']
        in_roi=select_poly.contains(this_point)
        
        if in_roi:
            tx="%s"%select_cell['cellID']
            #text.set_text(tx2)
                
#             select_x,select_y = select_poly.exterior.xy     
#             ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
#             ax.axis('off')
            
#             toTrack_roiIndex.append(select_index)
#             toTrack_roiFrame.append(max_frames)
#             print(select_index, this_point)
            
            #break
    text.set_text(tx)



this_cell=cells_tracked[1][172]
this_frame=1
global this_frame_polys
this_frame_polys=get_frame_polys(cells_tracked,this_frame)

next_frame_polys=get_frame_polys(cells_tracked,this_frame+1)
#fig=to_correct_plots[this_frame]
#fig2=plot_this_cells2(this_cell['cellID'],this_cell['trackedBy_next_frame'],cells_tracked)
matchID=this_cell['cellID']
cellID_list=this_cell['trackedBy_next_frame']
local_cells=cells_tracked.copy()
if(1):
    fig=plt.figure(figsize=(15,6))
    #fig, ax = plt.subplots(1, 2, )
    ax0=fig.add_subplot(121)
    ax1=fig.add_subplot(122)
    #ax = plt.axes()
    ax0.set_aspect('equal')
    ax1.set_aspect('equal')
  # plt.xlim(0,640)
   # plt.ylim(0,512)
#     #plt.axis('off')
#     ax[0].xlim(0,640)
#     ax[0].ylim(0,512)
#     ax[1].xlim(0,640)
#     ax[1].ylim(0,512)
#     
    ax0.clear()
    ax1.clear()
    text=ax0.text(0,0, "", va="bottom", ha="left")
    matchID=matchID.split('.')
    frame=int(matchID[0])
    index=int(matchID[1])
    this_cell=local_cells[frame][index]
    this_poly=this_cell['roiPoly']
    #print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
    #patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.3, zorder=2)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.7, zorder=2)
    
    ax0.add_patch(patch)
    patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=1)
    ax1.add_patch(patch)
    this_axis=this_cell['axis']
    linexy=np.array(this_axis)
    linex=linexy[:,0]
    liney=linexy[:,1]
    ax1.plot(linex,liney,'r*',alpha=0.5)
    ax0.plot(linex,liney,'c-',alpha=0.5)
    
    for this_cell in local_cells[frame]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax0.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax0.plot(linex,liney,'y-',alpha=0.5)
    
    
    for this_cell in local_cells[frame+1]:
        this_poly=this_cell['roiPoly']
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        ax1.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax1.plot(linex,liney,'y-',alpha=0.5)
    
    
    
    for this_id in cellID_list:
        this_id=this_id.split('.')
        frame=int(this_id[0])
        index=int(this_id[1])
        this_cell=local_cells[frame][index]
        this_poly=this_cell['roiPoly']
       # print(frame,index,this_cell['RelInt'],this_cell['GFP'],this_cell['DsRed'])    
        patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0.2,0.2,0.2], alpha=0.7, zorder=3)    
        
        
        
        ax1.add_patch(patch)
        this_axis=this_cell['axis']
        linexy=np.array(this_axis)
        linex=linexy[:,0]
        liney=linexy[:,1]
        ax1.plot(linex,liney,'c-',alpha=0.5)
    

    
    
    ax0.set_title('frame: %s cellID: %s'%(frame-1,matchID))
    ax1.set_title('frame: %s cellIDs: %s'%(frame,cellID_list))    
    

cid = fig.canvas.mpl_connect('button_press_event', onclickSelect)


In [ ]:
%matplotlib notebook
#import numpy as np
#import matplotlib.pyplot as plt
def onclick(event):
    tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
    text.set_text(tx)


fig = plt.figure()
ax = fig.add_subplot(121)
ax1 = fig.add_subplot(122)

ax.plot(np.random.rand(10))
text=ax.text(0,0, "", va="bottom", ha="left")
#ax.axis('off')

cid = fig.canvas.mpl_connect('button_press_event', onclick)

### Draw ROIs + Tracked Cells

In [ ]:
%matplotlib inline 
max_frames=len(cells_tracked)
tracked_plots=[]
for tracked_frame in range(1,max_frames+1):
    this_fig=draw_cell(cells_tracked,tracked_frame)
    tracked_plots.append(this_fig)


In [ ]:
%matplotlib inline 




this_tracked_plots=tracked_plots
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1))
interactive_plot    



In [ ]:
pwd()

### Save corrected cells

In [ ]:
#file_cells=os.listdir(dirNameFRAMECELLS)    
#file_cells.sort()
#for frame, this_file in enumerate(file_cells):
#    this_cells=cells_tracked[frame] 
#    fileNameFrameCells="%s%s.pkl"%(dirNameTRACKEDFRAMECELLS,os.path.splitext(os.path.basename(this_file))[0])   
#    save_cells(this_cells, fileNameFrameCells)

### Export ROIs + Tracked Cells

In [ ]:
#for this_frame, this_file in enumerate(fileROIs[0:max_frames+1]):
#    fileNameTrackedCells="%s%s.png"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])  
#    export_frame(cells_tracked, this_frame, dirNameROIs, fileROIs, fileNameTrackedCells)